In [18]:
# Importing packages

from rdflib import Graph, Literal, Namespace, URIRef, RDF, RDFS
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import math
import os
import pandas as pd

endpoint_url = "https://fokgsw.data.dice-research.org/sparql"
sparql = SPARQLWrapper(endpoint_url)

In [2]:
# creating a graph using classHierarchy.nt and reference-kg.nt
graph = Graph()

graph.parse("classHierarchy.nt", format="nt")
graph.parse("reference-kg.nt", format="nt")

<Graph identifier=N609c816625ee4040913492f02b55cb1f (<class 'rdflib.graph.Graph'>)>

In [3]:
def prune(graph):
    """
    Removes all the literals from the graph.
    """
    filtered_graph = Graph()
    for s , p, o in graph:
        if not isinstance(o, Literal):
            filtered_graph.add((s,p,o))
    return filtered_graph

In [4]:
graph = prune(graph)
len(graph)

660000

In [5]:
# s = '<http://rdf.freebase.com/ns/m.016srn>'
# p = '<http://rdf.freebase.com/ns/people.person.profession>'
# o = '<http://rdf.freebase.com/ns/m.01d_h8>'

In [6]:
def d_r_uri(graph, predicate_uri):
    """
    Retrives the domain and range a predicate
    """
    query = f"""
        SELECT ?d ?r
        WHERE {{
            {predicate_uri} rdfs:domain ?d .
            {predicate_uri} rdfs:range ?r .
        }}
    
    """
    domain, rrange = list(graph.query(query))[0]
    
    return domain, rrange
    
def init_query(graph: graph, predicate_uri: str):
    """
    Retrives the Domain Count, Range Count, Count of triples,
    Domain and range.
    """
    #Figuring out domain and range
    domain, rrange = d_r_uri(graph, predicate_uri)

    
    #counting D(p), R(p) and No. of triples with p as predicate
    c_dp = len(list(graph.triples((None, RDFS.domain, domain))))
    c_rp = len(list(graph.triples((None, RDFS.range, rrange))))
    c_triples = len(list(graph.triples((None, URIRef(p[1:-1]), None))))

    return c_dp, c_rp, c_triples, domain, rrange

In [7]:
#Path discovery

def generate_query_template(j):
    """
    Generates SPARQL templates to be used during fetching the path.
    Args:
        j: length of the path.
    eg:
        for j = 1
            SELECT ?p1
            WHERE{
                ?v0 ?p1 ?v1 .
            }
        for j = 2
            SELECT ?p1 ?p2
            WHERE{
                ?v0 ?p1 ?v1 .
                ?v1 ?p2 ?v2 .
            }
    """
    query_templates = []

    for i in range(j):
        select = "SELECT"
        triples = ""
        for k in range(i+1):
            select += f" ?p{k+1}"
            triples += f"\t?v{k} ?p{k+1} ?v{k+1} .\n\t"
        query = f"""
        {select}
        WHERE {{
            {triples}
        }}
        """
        query_templates.append(query)
    
    return query_templates

def prune_result(results):
    """
    Removes all the basic predicates, such as subclass, range, domain and type
    from the specific paths retrived
    Args:
        result: Result of the SPARQL query which contains the path
    """
    predicates = []
    terminology_predicates = [RDFS.subClassOf, RDFS.range, RDFS.domain, RDF.type]
    for result in results:
        if not any(i in terminology_predicates for i in result ):
            predicates.append(list(result))

    
    return predicates


In [8]:
#scoring

def calc_npmi(cnt_d,cnt_r, c_triples, qcnt,cntJ, p_epsilon=1e-18):
    """
    Calculates the NPMI value
    Args:
        cnt_d: count of domain
        cnt_r: count of range
        c_triples: count of triples
        cntJ: number of matches
    """
    tx_ty = cnt_d*1.0*cnt_r
    Pqp = cntJ/tx_ty
    if Pqp == 0 or Pqp == 1:
        Pqp = p_epsilon
    Pq = qcnt/tx_ty
    Pp = c_triples/tx_ty
    
    return math.log(Pqp/(Pq*Pp))/-math.log(Pqp)

def find_path(p,path_p,cnt_d, cnt_r, c_triples, domain, rrange,qcnt = None, qmin = 1):
    """
    Finds the path from a subject which should have a domain of target predicate, and
    the final object which should have a range of target predicate.
    Args:
        p: target predicate
        path_p : a list of predicates [the path we'll take]
        cnt_d: count of domain
        cnt_r: count of range
        c_triples: count of triples
        domain: domain of target predicate
        rrange: range of target predicate
    """
    if len(path_p) == 1:
        qry =f"""
        SELECT (COUNT(*) AS ?cnt)
        WHERE{{
            ?s a <{domain}> .
            ?s <{path_p[0]}> ?o .
            ?o a <{rrange}> .
        }}
        """
        qryJ = f"""
        SELECT COUNT(*) AS ?cnt SUM(?flg) AS ?matched sum(1-?flg) AS ?Notmatched
        WHERE{{
            ?s {p} ?o .
            ?s a <{domain}> .
            ?o a <{rrange}> .
            ?s <{path_p[0]}> ?o1 .
            ?o1 a <{rrange}> .
            bind((?o=?o1) AS ?flg)
        }}
        """
    else:
        qry = f"""
        SELECT COUNT(*) AS ?cnt
        WHERE{{
            ?s a <{domain}> .
            ?s <{path_p[0]}> ?o1 .
            ?o1 <{path_p[1]}> ?o2 .
            ?o2 a <{rrange}> .
        }}
        """
        qryJ = f"""
        SELECT COUNT(*) AS ?cnt SUM(?flg) as ?matched sum(1-?flg) AS ?Notmatched
        WHERE{{
            ?s {p} ?o .
            ?s a <{domain}> .
            ?o a <{rrange}> .
            ?s <{path_p[0]}> ?o1 .
            ?o1 <{path_p[1]}> ?o2 .
            ?o2 a <{rrange}> .
            bind((?o=?o2) AS ?flg)
        }}
        """
        

    if qcnt == None:
        sparql.setQuery(qry)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()['results']['bindings'][0]
        qcnt = int(results['cnt']['value'])
#         print("ran qcnt",end='->')

    if qcnt!= None and qcnt>qmin:
        sparql.setQuery(qryJ)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()['results']['bindings'][0]
#         print("qcnt>min",end='->')


        #calc_npmi
    if qcnt>qmin:
        cntJ = 0 if 'matched' not in results.keys() else int(results['matched']['value'])
#         print("cntJ")

        #print(f"Cd:{cnt_d} Cr:{cnt_r} Ct:{c_triples} qcnt:{qcnt} cntJ{cntJ}")
        npmi = calc_npmi(cnt_d,cnt_r, c_triples, qcnt,cntJ)
        
        return npmi
        

In [9]:
def main(s,p,o):
    """
    Main control function.
    Args:
        s: target subject
        p: target predicate
        o: target object
    """
    
    # Retrieving the counts
    c_dp, c_rp, c_triples, domain, rrange = init_query(graph, p)
#     print("Counting Done")
    
    #path finding
    k = 2
    z, q = [],[]

    for j in range(1,k+1):
        query_templates = generate_query_template(j)
        for query_template in query_templates:
            query = query_template.replace("?v0",f"{s}")
            query = query.replace(f"?v{j}", f"{o}")
    #         print(f"Query {j} --->\n{query}")
            results = graph.query(query)
            p_results = prune_result(results)

            for result in p_results:
                q.append((query_template, result))
#     print("Path finding done")
    

    #path scoring
    z = []
    for i in q[:200]:
        z.append(find_path(p, i[1],c_dp,c_rp,c_triples, domain, rrange))
#     print("Path scoring done")
    
    #final score calculation
    t = 1
    for c in z:
        if c is not None:
            t = t*(1-c)

    return round(1-t)
#     print("final score done")

    

In [10]:
# Creating a train graph, to retrive the triples in train.

def train():
    """
    Running the COPAAL approach on train
    """
    print("running training file")
    target_file = "fokg-sw-train-2023.ttl"
    
    train = Graph()
    train.parse(target_file, format="ttl")
    query = """
    SELECT ?st ?tv ?s ?p ?o
    WHERE{
        ?st a rdf:Statement .
        ?st ns1:hasTruthValue ?tv .
        ?st rdf:subject ?s .
        ?st rdf:predicate ?p .
        ?st rdf:object ?o .
    }
    """
    # Storing the triples in a list
    train_fc = []
    result = train.query(query)
    for row in result:
        train_fc.append([ row['st'], float(row['tv']), str(row['s']), str(row['p']), str(row['o'])])
    
    # Creating a dataframe of triples from train with predicted veracity

    if not os.path.exists("data.csv"):
        columns = ["statement", "Tvalue", "subject","predicate","object","Pred"]
        df = pd.DataFrame(columns=columns)
        df.to_csv("data.csv", index=False)

    #read last entry
    df = pd.read_csv('data.csv')
    if len(df) != 0:
        index = df.index[-1]
    else:
        index = -1
    #print(index)

    tp=0
    fp=0
    tn=0
    fn=0
    preds = []
    for i in range(index+1, len(train_fc)):
        s = f"<{train_fc[i][2]}>"
        p = f"<{train_fc[i][3]}>"
        o = f"<{train_fc[i][4]}>"
    #     print(s,p,o)
        pred = main(s,p,o)

        preds.append(pred)
        if train_fc[i][1] == 1 and pred == train_fc[i][1]:
            tp += 1
        elif train_fc[i][1] == 0 and pred == train_fc[i][1]:
            tn += 1
        elif train_fc[i][1] == 1 and pred != train_fc[i][1]:
            fn += 1
        elif train_fc[i][1] == 0 and pred != train_fc[i][1]:
            fp += 1

        print(f"Percentage ===>  {((i+1)/len(train_fc))*100}")
        row = {"statement": train_fc[i][0],
               "Tvalue"   : train_fc[i][1],
               "subject"  : train_fc[i][2],
               "predicate": train_fc[i][3],
               "object"   : train_fc[i][4],
               "Pred"     : pred}

        df.loc[len(df)] = row
        df.to_csv('data.csv', index=False)

    struct = f"""
              | Pred. +ve  | Pred. -ve  |
              ---------------------------
     Act. +ve |  TP {tp}   |  FN {fn}   |
     Act. -ve |  FP {fp}   |  TN {tn}   |
    """
    print(struct)



5000

In [ ]:
def test():
    """
    Running the COPAAL approach on train
    """
    print("running testing file")

    target_file = "fokg-sw-test-2023.ttl"
    
    test = Graph()
    test.parse(target_file, format="ttl")
    
    query = """
    SELECT ?st ?s ?p ?o
    WHERE{
        ?st a rdf:Statement .
        ?st rdf:subject ?s .
        ?st rdf:predicate ?p .
        ?st rdf:object ?o .
    }
    """
    
    # Storing the triples in a list
    test_fc = []
    result = test.query(query)
    for row in result:
        test_fc.append([ row['st'], float(row['tv']), str(row['s']), str(row['p']), str(row['o'])])
    
    
    if not os.path.exists("data.csv"):
        columns = ["statement", "subject","predicate","object","Pred"]
        df = pd.DataFrame(columns=columns)
        df.to_csv("data.csv", index=False)

    #read last entry
    df = pd.read_csv('data.csv')
    if len(df) != 0:
        index = df.index[-1]
    else:
        index = -1
    #print(index)

    for i in range(index+1, len(test_fc)):
        s = f"<{test_fc[i][2]}>"
        p = f"<{test_fc[i][3]}>"
        o = f"<{test_fc[i][4]}>"
    #     print(s,p,o)
        pred = main(s,p,o)

        print(f"Percentage ===>  {((i+1)/len(test_fc))*100}")
        row = {"statement": train_fc[i][0],
               "subject"  : train_fc[i][1],
               "predicate": train_fc[i][2],
               "object"   : train_fc[i][3],
               "Pred"     : pred}

        df.loc[len(df)] = row
        df.to_csv('data.csv', index=False)


In [ ]:
def convert2ttl(filename):
    
    df = pd.read_csv("data.csv")
    t_graph = Graph()
    predicate = URIRef("http://swc2017.aksw.org/hasTruthValue")
    for index, row in df.iterrows():
        iri = URIRef(row['statement'])
        pred = URIRef(row['Pred'])
        pred = Literal(pred, datatype=URIRef("http://www.w3.org/2001/XMLSchema#double"))
        t_graph.add((iri,predicate,pred))

    t_graph.serialize(destination=filename, format="turtle")
    


In [11]:
if __name__ == "__main__":
    print("In case of HTTP error please run this once more, it'll resume from the last stop!")
    file = input("Would you like to run the code for train or test?\nPlease Type train or test")
    if file == "test":
        test()
        convert2ttl("result_test.ttl")
    else:
        train()

Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt

ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ


ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->ran qcnt->Path scoring done
Percentage ===>  57.3
Counting Done
Path finding done
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->Path scoring done
Percentage ===>  57.4
Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->Path s

ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->qcnt>min->cntJ
Path scoring done
Percentage ===>  58.199999999999996
Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
Path scoring done
Percentage ===>  58.3
Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcn

ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
Path scoring done
Percentage ===>  58.9
Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
Path scoring done
Percentage ===>  59.0
Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->r

...

ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ


ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ


ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ


ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
Path scoring done
Percentage ===>  74.1
Counting Done
Path finding done
ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
Path scoring done
Percentage ===>  74.2
Counting Done
Path finding done
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->ran qcnt->ran qcnt->r

ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ
ran qcnt->qcnt>min->cntJ


In [ ]:
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = ",accuracy)

In [ ]:
main('<http://rdf.freebase.com/ns/m.09c7w0>', '<http://rdf.freebase.com/ns/location.location.contains>', '<http://rdf.freebase.com/ns/m.017wh>')